In [1]:
from numpy import float32, pi, imag, real, mean
# import matplotlib.pyplot as plt
from bokeh.plotting import Session, show, figure, output_server, hplot
from bokeh.models import Range1d
from bokeh.charts import Histogram

from gnss import codes
from gnss.receiver import channels
from gnss.signals import Signal
from gnss.receiver import sources
from gnss.filters import iir_filter
from gnss.acquisition import coarse, fine
from gnss.receiver import outputs
from gnss.tracking import tracking
from gnss.visualization import TrackingPlot, TrackingMultiPlot, CorrelationGridPlot, CorrelationLinePlot, FinePhasePlot

# DON'T FORGET
Run
    
    > bokeh-server -m --ip <ip>

from the command line.

In [2]:
session = Session(root_url='http://192.168.3.137:5006/', load_from_config=False)

In [3]:
# session.register('anon', '1234')

In [4]:
session.login('anon', '1234')
output_server('tracking', session=session)

In [5]:
%reload_ext autoreload
%autoreload 2


We are using data from the USRP front ends. This data contains *signed 4-bit complex* samples.


- L2:
    - 5MHz sampling frequency
    - 1.2276GHz center frequency
- L1:
    - 25MHz sampling
    - 1.57542GHz center frequency
- L5:
    - 25MHz sampling
    - 1.17645GHz center frequency

HIGH RATE DATA

- 0Hz IF
- 100MHz sampling frequency

- high rate center frequency
    - L1: 1.5675GHz
    - L2/L5 band: 1.212GHz

- high rate L1
    - 100MHz sampling frequency
    - 7.92MHz intermediate frequency

- high rate L2/L5
    - L1 7.92MHz intermediate frequency
    - L2 15.6MHz intermediate frequency
    - L5 -35.55MHz intermediate frequency

- high rate GLONASS
    - L1 34.5MHz intermediate frequency
    - L2 34MHz intermediate frequency

- high rate Galileo
    - E1BC 7.92MHz intermediate frequency
    - E5a -35.55MHz intermediate frequency
    - E5b -4.86MHz intermediate frequency

- high rate Beidou
    - B1 -6.402MHz intermediate frequency
    - B2 -4.86MHz intermediate frequency
    

First file: anything in the L1 band

Second file: anything in the low band (L2/L5)

These files are large, so we should choose a reasonable buffer size for their data. 2 seconds worth of data would be:

In [8]:
5e6 * 2, 25e6 * 2

(10000000.0, 50000000.0)

which are 10 mega-samples and 50 mega-samples respectively.

In [9]:
# filepath = '/media/DataStore/x300_l1_test_100MHz.sc4'
# source = sources.FileSource(filepath, file_f_samp=100e6, f_center=1.5675e9, buffer_size=25e6, bit_depth=4, real=False, decimation=2)
# 27min 47sec, fs: 5MHz L1/L2, 25MHz L5  OHz IF
filepath = '/mnt/gluster/by-drives/Dell/MU-Portable/OUDC3/US_A1_20140808_122714_002747_GPSL1_USRP1.dat'
source = sources.FileSource(filepath, file_f_samp=5e6, f_center=1.57542e9, buffer_size=5e6, bit_depth=4, real=False, decimation=1)

1667.699
we have 1.0 seconds of data


In [10]:
c_acq_plot = CorrelationLinePlot()
f_acq_plot = FinePhasePlot()
show(hplot(c_acq_plot.plot, f_acq_plot.plot))

In [11]:
step = 5e-3
f_update = 1. / step
block_length = 5e-3
block_size = block_length * source.f_samp
output_buff_size = source.buffer_size / (step * source.f_samp)

In [12]:
i_corr_plot = TrackingPlot('i corr', output_buff_size)
q_corr_plot = TrackingPlot('q corr', output_buff_size)
phase_plot = TrackingPlot('phase error', output_buff_size)
doppler_plot = TrackingPlot('doppler', output_buff_size)
show(hplot(i_corr_plot.plot, q_corr_plot.plot, phase_plot.plot, doppler_plot.plot))

In [31]:
source.seek(75)
source.load()
print('we have {0} seconds of data'.format(source.buffer_size / source.f_samp))

1667.699
we have 1.0 seconds of data


In [32]:
svid = 29
signal = Signal.GPSL1CA(svid)

c_acq = coarse.CoarseAcquirerLowMem(source, 5e-3, 2)
f_acq = fine.FineAcquirer(source, 2e-3, 1e-3, 29)

In [33]:
c_acq.acquire(signal)
chip0 = c_acq.chip
f_dopp = c_acq.f_dopp
time = c_acq.time
cn0 = c_acq.cn0
print(chip0, f_dopp, time, cn0)
c_acq_plot.update(c_acq)

836.6094 -2400.0 75.0 96.3029877809


In [34]:
f_acq.acquire(signal, time, chip0, f_dopp)
f_dopp = f_acq.f_dopp
time = f_acq.time
print(chip0, f_dopp, time)
f_acq_plot.update(f_acq)

836.6094 -2393.30357417 75.0
0.0 0.056 -0.160533126397 1.16849511223


In [35]:
dll_filter = iir_filter.FirstOrderLowpass(10, f_update)
pll_filter = iir_filter.FirstOrderLowpass(12, f_update)
# pll_filter = iir_filter.SecondOrderLowpass(omega_n=5 * 12, zeta=1.7, fs=f_update)
# pll_filter = iir_filter.SecondOrderLowpassV2(omega_n=5 * 32, zeta=1.7, fs=f_update)
fll_filter = iir_filter.FirstOrderLowpass(20, f_update);  fll_filter.f(0);  fll_filter.f(0)

delay = .5
correlator = tracking.Correlator(chip_delays=[-delay, 0., delay])

In [42]:
# source.reset()
time = f_acq.time
source.seek(time)
source.load()
theta = last_phase_error = 0.
chip = chip_out = chip0

1667.699


In [43]:
while True:
    while time < source.max_time - 2 * block_length:
        code_corr, dopp_corr = correlator.correlate(signal, source, block_size, time, chip, f_dopp, theta)
        early, prompt, late = code_corr
        chip_error = tracking.delay_discriminator(early, prompt, late)
        phase_error = -tracking.costas_discriminator(prompt)  # minus sign used b/c phase is subtracted in signal correlator
        chip_error = dll_filter.f(chip_error)
        phase_error = pll_filter.f(phase_error)
        dopp_error = -(phase_error - last_phase_error) / (step * 2 * pi)
        last_phase_error = phase_error
        dopp_error = fll_filter.f(dopp_error)
        # error correction
        chip -= chip_error
        theta -= phase_error
#         f_dopp -= dopp_error
        # carrier-aiding and propagation
        time += step
        f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
        chip += step * f_chip
        f_inter = signal.f_carrier - source.f_center
        theta += step * (f_inter + f_dopp) * 2. * pi
        # outputs
        chip_out = chip_out - chip_error
#         track_plot.push_data(time, real(prompt), imag(prompt), phase_error, f_dopp)
        i_corr_plot.push_data(time, real(prompt))
        q_corr_plot.push_data(time, imag(prompt))
        phase_plot.push_data(time, phase_error)
        doppler_plot.push_data(time, f_dopp)
    data = phase_plot.data.get()
    print(mean(data) / (2 * pi) / step)
    f_dopp -= mean(data) / (2 * pi) / step
    i_corr_plot.update()
    q_corr_plot.update()
    phase_plot.update()
    doppler_plot.update()
    source.advance(overlap=int(2 * source.f_samp * 0.01))

-1.52630279099
-0.478473227786
-0.695721777839
-0.505033339487
-0.342860608114
-0.26187128017
-0.144474891042
-0.0289443394429
0.0110016491737
0.08445603301
0.156556593014
0.263905746373
0.418758203774
0.475153139574
0.656236226134
0.833986276882
0.855242103551
0.896125486198
0.933474245523
0.963577985804
1.06379901142
1.17101514141
1.16766769859
1.23026854128
1.24996467739
1.22679730117
1.25829982379
1.25620857955
1.28126189961
1.24798564335
1.24703791791
1.17932092011
1.12395977777
1.03946996614
0.947028109651
0.887685595318
0.835250362857
0.770552281267
0.686230072351
0.620594578862
0.526697965506
0.431227111384
0.284300543688
0.239613465501
0.156871752495
0.109571291199
0.0388134235216
-0.0357004960333
-0.0948294424829
-0.144287976227
-0.186424722693
-0.195110347378
-0.263789651725
-0.273131676812
-0.314525097431
-0.309089226348
-0.287219357215
-0.294671174621
-0.345558079868
-0.383780834683
-0.41331989484
-0.430543623579


KeyboardInterrupt: 

.

.

PARTIAL TRACKING

In [25]:
# time = 0.0
# source.reset()
time = f_acq.time

In [19]:
theta = last_phase_error = 0.
chip = chip_out = chip0
while time < source.max_time - 2 * block_length:
    code_corr, dopp_corr = correlator.correlate(signal, source, block_size, time, chip, f_dopp, theta)
    early, prompt, late = code_corr
    chip_error = tracking.delay_discriminator(early, prompt, late)
    phase_error = -tracking.costas_discriminator(prompt)  # minus sign used b/c phase is subtracted in signal correlator
    chip_error = dll_filter.f(chip_error)
    phase_error = pll_filter.f(phase_error)
    dopp_error = (phase_error - last_phase_error) / (step * 2 * pi)
    last_phase_error = phase_error
    dopp_error = fll_filter.f(dopp_error)
    # error correction
    chip -= chip_error
    theta -= phase_error
    f_dopp -= dopp_error
    # carrier-aiding and propagation
    time += step
    f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
    chip += step * f_chip
    f_inter = signal.f_carrier - source.f_center
    theta += step * (f_inter + f_dopp) * 2. * pi
    # outputs
    chip_out = chip_out - chip_error
    track_plot.push_data(time, real(prompt), imag(prompt), phase_error, f_dopp)

In [20]:
# fig = output.plot(library='matplotlib')
# plt.show()
track_plot.update()

In [3]:
from numpy import arange
from numpy.random import rand
from bokeh.plotting import output_notebook, show
from bokeh.plotting import Plot, GridPlot
from bokeh.models import ColumnDataSource, DataRange1d
from bokeh.models.glyphs import Line

output_notebook()
time = arange(20)
y = rand(len(time))
data = ColumnDataSource(data=dict(time=time, y=y))
x_range = DataRange1d(sources=[data.columns('time')])
y_range = DataRange1d(sources=[data.columns('y')])
plot = Plot(x_range=x_range, y_range=y_range, title='data', plot_width=250, plot_height=250)
# plot = Plot(x_range=None, y_range=None, title='data', plot_width=250, plot_height=250)  # < ahh needs x_range/y_range!
plot.add_glyph(data, Line(x='time', y='y'))
show(plot)